In [ ]:
pip install spacy


In [ ]:
pip install textblob


In [ ]:
pip install python -m spacy download fr_core_news_sm

In [ ]:
pip install pandas

In [ ]:
pip install nltk

In [ ]:
pip install aiogoogletrans

In [ ]:
pip install unidecode

In [ ]:
pip install emoji

In [ ]:
!python -m spacy download fr_core_news_sm

In [ ]:
from __future__ import unicode_literals
from unidecode import unidecode
import emoji
from aiogoogletrans import Translator
import asyncio
import nltk
nltk.download('stopwords')
import re
import spacy
nlp = spacy.load("fr_core_news_sm")
french_stopwords = set(nltk.corpus.stopwords.words('french'))

1- get hashtags
2- get mentions
3- get urls
4- remove hashtags
5- treat mentions "remove only @"
6- replace emojis "only arabic case is treated"
7- text = re.sub('http\S+\s*', ' ', text)  # remove URLs
7- remove punctuations
10- remove stop words 
8- remove non used characters 
9- remove repeated characters
11- for arabic remove tashkeel
12- Strip tatweel from a text.
13- stem words
14- normalizeArabic
15- tokenize

get emails

In [127]:
def get_emails(text):
	emails = re.findall(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+\b)', text)
	return emails

get hashtags

In [128]:
def split_hashtag_to_words(tag):
    tag = tag.replace('#', '')
    tags = tag.split('_')
    if len(tags) > 1:
        return tags
    pattern = re.compile(r"[A-Za-z]+|\d+")
    return pattern.findall(tag)

def extract_hashtag(text):
    hash_list = ([re.sub(r"(\W+)$", "", i) for i in text.split() if i.startswith("#")])
    word_list = []
    for word in hash_list:
        word_list.extend(split_hashtag_to_words(word))
    return word_list


In [129]:
text = "this is #én_example #text "
print(extract_hashtag(text))

['én', 'example', 'text']


get mentions

In [130]:
def extract_mentions(text):
    mentions = re.findall(r'@(\w+)', text)
    return mentions

text_arabic = "@user1 say hi to @user2"
mentions_arabic = extract_mentions(text_arabic)
print(mentions_arabic) 


['user1', 'user2']


3- get urls

In [131]:
def extract_urls(text):
	urls = re.findall(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?',text)
	return urls

arabic_text_with_urls = "enter those sites to find more infos : https://example.com/index.php، https://example.org"
urls = extract_urls(arabic_text_with_urls)
print(urls)


[('https', 'example.com', '/index.php'), ('https', 'example.org', '')]


4- treat hashtags : remove the whole hashtag or just the character 

In [132]:
def clean_hashtag(text):
    words = text.split()
    cleaned_text = []
    for word in words:
        if not is_hashtag(word):
            cleaned_text.append(word)
    return " ".join(cleaned_text)

def is_hashtag(word):
    return word.startswith("#")


def remove_hashtag_char(text):
    hashtag_pattern = r'#+'
    cleaned_text = re.sub(hashtag_pattern, '', text)
    return cleaned_text

In [133]:
text_with_hashtag = "this text #must be cleaned #from_hashtags "
cleaned_text = clean_hashtag(text_with_hashtag)
print(cleaned_text)


this text be cleaned


In [134]:
text_with_hashtag = "this text #must be cleaned #from_hashtags "
cleaned_text = remove_hashtag_char(text_with_hashtag)
print(cleaned_text)

this text must be cleaned from_hashtags 


remove emails

In [135]:
def remove_emails(text):
	return re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+\b)',"",text)

In [136]:
text="this is example user@gmail.com"
remove_emails(text)

'this is example '

5- remove mentions 

In [137]:
def remove_mentions(text):
    cleaned_text = re.sub(r'@+', '', text)
    return cleaned_text

text_with_mentions = "hi @user1"
cleaned_text = remove_mentions(text_with_mentions)
print(cleaned_text)


hi user1


6- replace emojis "only arabic case is treated"

In [ ]:
with open('Selected_Fr_emojis.csv','r',encoding='utf-8') as f:
    lines = f.readlines()
    emojis_ar = {}
    for line in lines:
        line = line.strip('\n').split(';')
        emojis_ar.update({line[0].strip():line[1].strip()})


def remove_emoji(text):
    emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text


def emoji_native_translation(text):
    text = text.lower()
    loves = ["<3", "♥",'❤']
    smilefaces = []
    sadfaces = []
    neutralfaces = []

    eyes = ["8",":","=",";"]
    nose = ["'","`","-",r"\\"]
    for e in eyes:
        for n in nose:
            for s in ["\)", "d", "]", "}","p"]:
                smilefaces.append(e+n+s)
                smilefaces.append(e+s)
            for s in ["\(", "\[", "{"]:
                sadfaces.append(e+n+s)
                sadfaces.append(e+s)
            for s in ["\|", "\/", r"\\"]:
                neutralfaces.append(e+n+s)
                neutralfaces.append(e+s)
            #reversed
            for s in ["\(", "\[", "{"]:
                smilefaces.append(s+n+e)
                smilefaces.append(s+e)
            for s in ["\)", "\]", "}"]:
                sadfaces.append(s+n+e)
                sadfaces.append(s+e)
            for s in ["\|", "\/", r"\\"]:
                neutralfaces.append(s+n+e)
                neutralfaces.append(s+e)

    smilefaces = list(set(smilefaces))
    sadfaces = list(set(sadfaces))
    neutralfaces = list(set(neutralfaces))
    t = []
    for w in text.split():
        if w in loves:
            t.append("Amour")
        elif w in smilefaces:
            t.append("Sourire")
        elif w in neutralfaces:
            t.append("Neutre")
        elif w in sadfaces:
            t.append("Triste")
        else:
            t.append(w)

    newText = " ".join(t)
    return newText

def is_emoji(word):
    if word in emojis_ar:
        return True
    else:
        return False
    
def add_space(text):
    return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()

translator = Translator()
loop = asyncio.get_event_loop()
def translate_emojis(words):
    word_list = list()
    words_to_translate = list()
    for word in words :
        t = emojis_ar.get(word.get('emoji'),None)
        if t is None:
            word.update({'translation':'normale','translated':True})
            #words_to_translate.append('normal')
        else:
            word.update({'translated':False,'translation':t})
            words_to_translate.append(t.replace(':','').replace('_',' '))
        word_list.append(word)
    return word_list

def emoji_unicode_translation(text):
    text = add_space(text)
    words = text.split()
    text_list = list()
    emojis_list = list()
    c = 0
    for word in words:
        if is_emoji(word):
            emojis_list.append({'emoji':word,'emplacement':c})
        else:
            text_list.append(word)
        c+=1
    emojis_translated = translate_emojis(emojis_list)
    for em in emojis_translated:
        text_list.insert(em.get('emplacement'),em.get('translation'))
    text = " ".join(text_list)
    return text
    
def clean_emoji(text):
    text = emoji_native_translation(text)
    text = emoji_unicode_translation(text)
    return text

remove URLs

In [139]:
texte_avec_emojis = "Je t'aime ❤️😊"
texte_propre = clean_emoji(texte_avec_emojis)
print(texte_propre)


texte_avec_emojis = "Je suis très triste 😢"
texte_propre = clean_emoji(texte_avec_emojis)
print(texte_propre)


texte_avec_emojis = "Je suis de très bonne humeur 😄"
texte_propre = clean_emoji(texte_avec_emojis)
print(texte_propre)


je t'aime Coeur rouge blush
je suis très triste cri
je suis de très bonne humeur sourire sueur


In [140]:
def remove_urls(text):
    cleaned_text = re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?',"",text)
    
    return cleaned_text

In [154]:
text = "checké this http://www.google.com"
cleaned_text = remove_urls(text)
print(" text with URLs removed:", cleaned_text)

 text with URLs removed: checké this 


remove stopwords

In [142]:
def remove_french_stopwords(text):
  return " ".join([t for t in text.split() if t.lower() not in french_stopwords])


sample_text = "Ceci est une phrase d'exemple contenant des stop words français."
filtered_text = remove_french_stopwords(sample_text)
print("Texte sans stop words français:", filtered_text)


Texte sans stop words français: Ceci phrase d'exemple contenant stop words français.


7- remove punctuations 

In [ ]:
all_punctuation = string.punctuation

def remove_punctuations(text):
    return re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)

In [155]:
text = "remove those punctuations !;!'-"
cleaned_text = remove_punctuations(text)
print("text with punctuations removed:", cleaned_text)

text with punctuations removed: remove those punctuations      


8- remove non used characters (u can keep numbers)

In [145]:
def remove_non_french(text):
    french_pattern = re.compile(r'[^\s\dA-Za-zàâäéèêëîïôöùûüçÀÂÄÉÈÊËÎÏÔÖÙÛÜÇ]+')
    cleaned_text = french_pattern.sub('', text)
    
    return cleaned_text


In [156]:
text = "Bonjour! Comment ça va ? 123 Привет"
cleaned_text = remove_non_french(text)
print(cleaned_text)

Bonjour Comment ça va  123 


9- remove repeated characters

In [147]:
def remove_repeated_characters(text):
    repeated_pattern = re.compile(r'(\S)(\1)+', re.UNICODE)
    cleaned_text = repeated_pattern.sub(r'\1', text)
    return cleaned_text

In [157]:
text = "thissssé        isss theee "
cleaned_text = remove_repeated_characters(text)
print("Text with repeated characters removed:", cleaned_text)

Text with repeated characters removed: thisé        is the 


remove multiple spaces

In [149]:
def remove_multiple_spaces(text):
	return ' '.join(text.split())

stemming words

In [150]:
def tokenize_and_lemmatize(text):
    doc = nlp(text)
    tokens_lemmas = [token.lemma_ if token.lemma_ != "-PRON-" and token.lemma_ != "be" else token.text for token in doc]
    return tokens_lemmas

text = "Les renards bruns rapides sautent par-dessus les chiens paresseux"
tokens_lemmas = tokenize_and_lemmatize(text)
print(tokens_lemmas)


['le', 'renard', 'brun', 'rapide', 'sauter', 'par', '-', 'dessus', 'le', 'chien', 'paresseux']


the proposed algorithm to treat the text

In [151]:
text="ce sont des exemples de la langue français"
emails = get_emails(text)
print(emails)
hashtags = extract_hashtag(text)
print(hashtags)
mentions = extract_mentions(text)
print(mentions)
urls = extract_urls(text)
print(urls)

[]
[]
[]
[]


In [152]:
def clean_text(text):
    text=remove_emails(text)
    text=clean_hashtag(text)
    text=remove_mentions(text)
    text=remove_urls(text)
    text=clean_emoji(text)
    text=remove_punctuations(text)
    text=remove_repeated_characters(text)
    text=remove_multiple_spaces(text)
    text=remove_french_stopwords(text)
    return tokenize_and_lemmatize(text)


clean_text(text)

['exemple', 'langue', 'français']